In [2]:
import tiktoken
import openai
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers  import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


### 1. BM25 기반 검색기 만들기
- BM25는 사용자가 검색한 단어가 특정 문서 내에서 많이 등장할수록 (TF가 높을수록), 전체 문서 중 매우 적은 수의 문서에만 나타나는 단어일수록 (IDF가 클수록), 그리고 문서의 길이가 짧을수록, 더 높은 점수를 주어 해당 문서를 사용자가 원하는 검색 결과의 상위에 배치함

In [3]:
loader = PyPDFLoader('./gsat_170823.pdf')
pages = loader.load_and_split() # pdf 페이지 단위로 쪼깨서 읽어 들임
print(len(pages))

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 75 65536 (offset 0)
Ignoring wrong pointing object 85 65536 (offset 0)
Ignoring wrong pointing object 94 65536 (offset 0)
Ignoring wrong pointing object 98 65536 (offset 0)
Ignoring wrong pointing object 102 65536 (offset 0)
Ignoring wrong pointing object 109 65536 (offset 0)
Ignoring wrong pointing object 119 65536 (offset 0)
Ignoring wrong pointing object 132 65536 (offset 0)
Ignoring wrong pointing object 148 65536 (offset 0)
Ignoring wrong pointing object 176 65536 (offset 0)
Ignoring wrong pointing object 188 65536 (offset 0)
Ignoring wrong pointing object 192 65536 (offset 0)
Ignoring wrong pointing object 199 65536 (offset 0)
Ignoring wrong pointing object 203 65536 (offset 0)
Ignoring wrong pointing object 207 65536 (offset 0)
Ignoring wrong pointing object 211 65536 (offset 0)
Ignoring wrong pointing object 215 65536 (offset 0)
Ignoring wrong pointing object 219 65536 (offset 0)
Ignoring wrong poi

27


In [4]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

C:\Users\yechan_amd\AppData\Local\Temp\ipykernel_21516\1537676741.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')


In [4]:
pages[0]

Document(metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': './gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등\nIM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품과 구성비율은 HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등 54%, TV, 모니터, 냉장

In [5]:
pages[0].page_content

'2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등\nIM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품과 구성비율은 HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등 54%, TV, 모니터, 냉장고, 세탁기, \n에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 \n∙ 지역별로는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 \n지역총괄의 생산ㆍ판매법인 등 165개의 동종업종을 영위하는 종속기업으로 구성\n∙ 국내사업장 : 수원, 구미, 기흥, 화성, 온양, 광주\n∙ 해외사업장 : 86개국 220개지점 (15년 말 기준)\n1) 연결재무재표 기준\n2) 연결재무재표 기준'

In [6]:
pages[1].page_content

'3\n2) 주요 사업 분야 \n⑴ CE(Consumer Electronics)부문 \n① 산업의 특성 \n∙TV산업은 1926년 흑백 TV 개발, 1954년 RCA사가 Color TV(21") 양산/판매를 시작한 이래로 트리니트\n론 브라운관(1967년), 완전평면 브라운관(1996년) 개발 등 기술적인 발전을 거듭\n∙ 주요 국가 보급률이 90% 넘어서면서 브라운관 TV사업의 성장은 정체 \n∙ 그러나 하드웨어 측면에서의 Flat Panel TV(LCD, PDP) 출시, 소프트웨어 측면에서의 디지털 방송 \n확산(영/미 1998년~ )을 통해 TV 시장은 또 다시 강력한 성장 모멘텀을 되찾음 \n∙FPTV는 화질, 디자인 등 제품 성능의 향상과 지속적인 Set가격 하락으로 인해 성장을 지속하며 기존 \nCRT 시장을 빠르게 대체\n∙ 2010년 3D TV출시\n∙ 2011년~2012년 스마트TV 태동 \n∙ 2013년 UHD TV 출시 \n∙ 2014년 Curved TV 출시 \n∙2016년도 전체 TV 수요는 전년비 1.6% 하락한 2억 2,252만대로 제품별로는 LCD-TV(LED-TV포함)가 \n전년비 1.3% 하락한 2억 2,131만대 전망되나, 고해상도 Needs확대에 따라 UHD TV는 56백만대로 \n전년比 75% 성장할 것으로 예상(출처 : 2016.9월 IHS Report)\n② 국내외 시장연건 \n∙ TV시장의 Mega Trend인 대형화/고화질화가 Device간, 업체간 경쟁 격화에 따라 더욱 빠른 속도로 \n진행되고 있으며, 이에 따라 제품력과 브랜드파워를 앞세운 Major 업체의 강세가 지속\n∙ 고화질 및 슬림 제품에 대한 소비자 Needs가 높아짐에 따라 친환경 소재인 LED BLU(Back Light \nUnit)를 적용하여 TV의 밝기와 명암비를 높이고 소비전력을 낮춘 LED TV가 시장의 Main Stream으로 \n자리 잡음\n∙ 2010년에는 다양한 App을 이용할 수있는 스마트 TV를 출시하고 시장을 주도\n∙2012년

### Chunking하자!

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [8]:
texts

[Document(metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': './gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등'),
 Document(metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00'

In [9]:
len(texts)

69

In [10]:
len(texts[0].page_content)

490

### BM-25 검색기

In [11]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2

In [12]:
docs = bm25_retriever.get_relevant_documents("삼성전자의 사업 영역은?")

C:\Users\yechan_amd\AppData\Local\Temp\ipykernel_21516\1405877725.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = bm25_retriever.get_relevant_documents("삼성전자의 사업 영역은?")


In [13]:
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': './gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
12 Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ? A  삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를  인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에서 점유 율 1~2위를 달리고 있으며 매출도 상당합니다. 이번 삼성전자의 하만 인수는 모바일 및 가전 시장에서  나아가 Connected Car 시장 진입에 진출했다는 것에 큰 의의가 있습니다. 향후 10년 시장 내 Connected  Car의 비중은 90%에 육박할 전망입니다. LG전자 역시 자동차부품(VC) 사업 부문에 있어 상당한 경쟁  위협 요인가 될 것으로 내다봤습니다. <관련기사> 전장사업 강자 하만과 손잡은 삼성…스마트카 ‘티어1’ 노린다 (한국경제. 2016-11-21) 삼성전자는 자동차 전장(電裝)사업에서 후발 주자다. 2005년 전장사업을 시작한 LG전자는
474
******************************
{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': './gsat_170823.pdf', 'total_pages': 27

### 임베딩 기반 검색기

In [15]:
chroma_vector = Chroma.from_documents(texts, model_huggingface)
chroma_retriever = chroma_vector.as_retriever(search_kwargs={'k':2})

In [16]:
docs = chroma_retriever.invoke("삼성전자의 사업 영역은?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'page': 9, 'creator': 'nPDF (pdftk 1.41)', 'page_label': '10', 'source': './gsat_170823.pdf', 'moddate': '2017-08-16T00:21:02-08:00', 'total_pages': 27, 'creationdate': '2017-08-16T00:21:02-08:00', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)'}
:
11 Ⅱ 기업 상세 분석 1  사업분야(내용) Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ? A  삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica- tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다. ⑴ Consumer Electronics (CE) 부문  ① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에  도전 ② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비 ③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발 ⑵ Information technology & Mobile communication (IM) 부문 ① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공
499
******************************
{'page_label': '10', 'total_pages': 27, 'source': './gsat_170823.pdf', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'pro

### BM-25 + 임베딩 앙상블 기반 검색기

In [17]:
ensemble_retriever = EnsembleRetriever(
                    retrievers = [bm25_retriever, chroma_retriever], weight = {0.5,0.5})

docs = ensemble_retriever.invoke("삼성전자의 사업 영역은?")

In [18]:
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'page': 9, 'moddate': '2017-08-16T00:21:02-08:00', 'source': './gsat_170823.pdf', 'total_pages': 27, 'creationdate': '2017-08-16T00:21:02-08:00', 'page_label': '10'}
:
11 Ⅱ 기업 상세 분석 1  사업분야(내용) Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ? A  삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica- tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다. ⑴ Consumer Electronics (CE) 부문  ① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에  도전 ② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비 ③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발 ⑵ Information technology & Mobile communication (IM) 부문 ① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공
499
******************************
{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': './gsat_170823